# Building devices in python

This tutorial demonstrates how you can easily build cells in python with gdsfactory

In [ ]:
import gdsfactory as gf

Let's start by creating a simple cell with a polygon on GDS layer (1, 0)

In [ ]:
c = gf.Component()
c.add_polygon([[0, 0], [1, 1], [2, 0]], layer=(1, 0))
c.plot()

Adding polygons is the most fundamental operation when creating a layout, but it's usually more convenient to reuse something we already have existing. Let's demonstrate adding some text, in a DRC-friendly rectangular font.

In [ ]:
c = gf.Component()
# text_rectangular() produces a cell. we place a reference of that cell in `c`
c << gf.c.text_rectangular("Hello Kyushu!")
c.plot()

There are many useful shapes and devices built into gdsfactory, in what we call the "generic PDK". Let's demonstrate a few of them.

In [ ]:
c = gf.Component()
c << gf.c.text_freetype("gdsfactory has lots of built-in shapes and devices")
circle = c << gf.c.circle(radius=30)
circle.move((-30, -60))
rect = c << gf.c.rectangle((30, 10), layer=(2,0))
rect.move((20, -60))
bezier = c << gf.components.bezier(control_points=[[0.0, 0.0], [30.0, 0.0], [50.0, 40], [30.0, -50]], npoints=201, with_manhattan_facing_angles=True, cross_section='xs_sc')
bezier.move((60, -60))
gc = c << gf.c.grating_coupler_elliptical()
gc.move((130, -60))
greek_cross = c << gf.c.greek_cross()
greek_cross.move((250, -60))
c.plot()

c.write_gds("ex1.gds")

When creating a cell (or pcell) you'd like to use in one of your designs, you should put it in a function with an `@cell` decorator, as shown below.

In [ ]:
@gf.cell
def my_cell(my_parameter: int = 1) -> gf.Component:
    c = gf.Component()
    c << gf.c.text_freetype(f"my parameter is {my_parameter}")
    return c

c = my_cell(my_parameter=3)
c.write_gds("ex2.gds")

When you open the GDS file written from the cell above, you should notice that the cell has been named automatically according to the function + parameter names. Furthermore, the caching mechanism in gdsfactory ensures that
- there are never naming conflicts between cells
- the same cell is never created twice, saving time during generation

In [ ]:
# these cells don't just look the same, they point to the same object in memory
my_cell(my_parameter=3) is my_cell(my_parameter=3)

## Ports
It's useful to define ports on your cells so that they can be connected together in higher-level cells and circuits.

In [ ]:
@gf.cell
def my_triangle():
    c = gf.Component()
    c.add_polygon([[0, 0], [1, 1], [2, 0]], layer=(1, 0))
    c.add_port(name="p1", center=(1, 1), orientation=90, cross_section="xs_sc")
    return c

my_triangle().plot()

In [ ]:
kissy_kissy = gf.Component()
t1 = kissy_kissy << my_triangle()
t2 = kissy_kissy << my_triangle()
t1.rotate(-90)
t2.connect("p1", t1.ports["p1"])
kissy_kissy.plot()